In [1]:
POS_DATA = [
    # code, name, linear_pos, pickup_time, place_time
    (0,  'ready', 1.0, 3, 3), 
    (1,  'f0',    0.9, 3, 3), 
    (2,  'f1',    0.8, 3, 3), 
    (3,  'f2',    0.7, 3, 3), 
    (4,  'f3',    0.6, 3, 3), 
    (5,  'f4',    0.5, 3, 3), 
    (6,  'f5',    0.4, 3, 3),
    (7,  'w0',    0.9, 3, 3),
    (8,  'w1',    0.8, 3, 3),
    (9,  'w2',    0.7, 3, 3),
    (10, 'w3',    0.6, 3, 3),
    (11, 'w4',    0.5, 3, 3),
    (12, 'w5',    0.4, 3, 3),
    (13, 'fin',   0.3, 3, 3), 
    (14, 'ret0',   0.2, 3, 3),
    (15, 'ret1',   0.2, 3, 3),
    (16, 'ret2',   0.2, 3, 3),
]

BASKET_DATA = [
    # code, name, cooking_time
    (0,  'no',                      0), 
    (1,  'empty',                   0), 
    (11, 'boneless_chicken',        6.5*60), 
    (12, 'bone_chicken',            9.5*60), 
    (13, 'potato',                  3.5*60), 
    (14, 'cheeseball',              4.5*60), 
    (21, 'cooked_boneless_chicken', 0),
    (22, 'cooked_bone_chicken',     0),
    (23, 'cooked_potato',           0),
    (24, 'cooked_cheeseball',       0),
]

ROBOT_STATUS = {"linear": 0, "holding" : 0}
BASKET_STATUS = dict([(n, 0) for _, n, *_ in POS_DATA])
FRYING_TIME = dict([('f{}'.format(i), 99999) for i in range(6)])
RECIPE = dict([(n, t) for c, n, t in BASKET_DATA if 10 < c < 20])

pos_cn = {code: name for code, name, pos, _, _ in POS_DATA }
pos_nc = {name: code for code, name, pos, _, _ in POS_DATA }
pos_cp = {code: pos for code, name, pos, _, _ in POS_DATA }

basket_cn = {code: name for code, name, _ in BASKET_DATA }
basket_nc = {name: code for code, name, _ in BASKET_DATA }

In [6]:
BASKET_STATUS['f1'] = basket_nc['cheeseball']
BASKET_STATUS['f2'] = basket_nc['cheeseball']

FRYING_TIME['f1'] = -25
FRYING_TIME['f2'] = 4.5
print(FRYING_TIME)
print()
print(BASKET_STATUS)
print()
print(pos_cn)

{'f0': 99999, 'f1': -25, 'f2': 4.5, 'f3': 99999, 'f4': 99999, 'f5': 99999}

{'ready': 0, 'f0': 0, 'f1': 14, 'f2': 14, 'f3': 0, 'f4': 0, 'f5': 0, 'w0': 0, 'w1': 0, 'w2': 0, 'w3': 0, 'w4': 0, 'w5': 0, 'fin': 0, 'ret0': 0, 'ret1': 0, 'ret2': 0}

{0: 'ready', 1: 'f0', 2: 'f1', 3: 'f2', 4: 'f3', 5: 'f4', 6: 'f5', 7: 'w0', 8: 'w1', 9: 'w2', 10: 'w3', 11: 'w4', 12: 'w5', 13: 'fin', 14: 'ret0', 15: 'ret1', 16: 'ret2'}


In [ ]:
from time import time
from time import sleep
import copy

def rt_status_update():
    prev_time = time()
    while True:
        sleep(0.05)

        for i in range(6):
            if BASKET_STATUS['f{}'.format(i)] > 10: 
                FRYING_TIME['f{}'.format(i)] -= time()-prev_time

                if FRYING_TIME['f{}'.format(i)] < 0 and BASKET_STATUS['f{}'.format(i)] < 20:
                    BASKET_STATUS['f{}'.format(i)] = BASKET_STATUS['f{}'.format(i)] + 10
            else:
                FRYING_TIME['f{}'.format(i)] = 9999

            if FRYING_TIME['f{}'.format(i)] < -30:
                print("'f{} overcooked {}".format(i, FRYING_TIME['f{}'.format(i)]))

        prev_time = time()
        PRE_BASKET_STATUS = copy.deepcopy(BASKET_STATUS)

rt_status_update()

In [ ]:
print(FRYING_TIME)
print(RECIPE)
print(BASKET_STATUS)

In [ ]:
class Command():
    def __init__(self, pos_code, category, motion_time = 0):
        self.cat = category
        self.pos_code = pos_code
        self.pos_name = pos_cn[pos_code]
        self.motion_time = motion_time

    def __repr__(self):
        return "< cmd {} >".format(self.cat + self.pos_code)

    def get_motion_time(self):
        return self.motion_time

    def get_cmd_addr(self):
        return self.cat + self.pos
    
    def done(self):
        pass

class CMD_GO(Command):
    def __init__(self, pos_code):
        super().__init__(pos_code, 0, 0)

    def get_motion_time(self, start_pos_code):
        if pos_cp[start_pos_code] == pos_cp[self.pos_code]:
            return 0
        else:
            return abs(pos_cp[self.pos_code] - pos_cp[start_pos_code]) + 1

    def done(self):
        ROBOT_STATUS["linear"] = self.pos_code

class CMD_PICKUP(Command):
    def __init__(self, pos_code, motion_time):
        super().__init__(pos_code, 100, motion_time)
    def done(self):
        ROBOT_STATUS["holding"] = BASKET_STATUS[self.pos_name]
        BASKET_STATUS[self.pos_name] = basket_nc['없음']
        for i in range(6):
            if self.pos_name == 'f{}'.format(i):
                FRYING_TIME[self.pos_name] = 0

class CMD_PLACE(Command):
    def __init__(self, pos_code, motion_time):
        super().__init__(pos_code, 200, motion_time)
    def done(self):
        BASKET_STATUS[self.pos_name] = ROBOT_STATUS["holding"]
        ROBOT_STATUS["holding"] = basket_nc['없음']
        
        for i in range(6):
            if self.pos_name == 'f{}'.format(i):
                FRYING_TIME[self.pos_name] = RECIPE[basket_cn[BASKET_STATUS[self.pos_name]]]

class CMD_SHAKE(Command):
    def __init__(self, pos_code):
        super().__init__(pos_code, 300, 50)

cmd_go = dict([(code, CMD_GO(code)) for code, *_ in POS_DATA])
cmd_pick_up = dict([(code, CMD_PICKUP(code, pick_time)) for code, _, _, pick_time, _ in POS_DATA])
cmd_place = dict([(code, CMD_PLACE(code, place_time)) for code, _, _, _, place_time in POS_DATA])
cmd_shake = dict([(code, CMD_SHAKE(code)) for code, *_ in POS_DATA])

class PickNPlace():
    def __init__(self, pick_pos_name, place_pos_name):
        self.pick_pos_code = pos_nc[pick_pos_name]
        self.place_pos_code = pos_nc[place_pos_name]
        self.cmds = [cmd_go[self.pick_pos_code], cmd_pick_up[self.pick_pos_code], cmd_go[self.place_pos_code], cmd_place[self.place_pos_code]]
    
    def __repr__(self):
        return "< pick {}, place {} >".format(pos_cn[self.pick_pos_code], pos_cn[self.place_pos_code])

    def get_time(self):
        time = (
            self.cmds[0].get_motion_time(ROBOT_STATUS['linear']) 
            + self.cmds[1].get_motion_time() +
            + self.cmds[2].get_motion_time(self.pick_pos_code)
            + self.cmds[3].get_motion_time()
        )
        return time

class PickNShake():
    def __init__(self, pick_pos_code, shake_pos_code):
        self.pick_pos_code = pos_nc[pick_pos_name]
        self.shake_pos_code = pos_nc[shake_pos_code]
        self.cmds = [cmd_go[self.pick_pos_code], cmd_pick_up[self.pick_pos_code], cmd_go[self.shake_pos_code], cmd_shake[self.shake_pos_code]]

    def __repr__(self):
        return "< pick {}, shake {} >".format(pos_cn[self.pick_pos_code], pos_cn[self.shake_pos_code])
    
    def get_time(self):
        time = (
            self.cmds[0].get_motion_time(ROBOT_STATUS['linear']) 
            + self.cmds[1].get_motion_time() +
            + self.cmds[2].get_motion_time(self.pick_pos_code)
            + self.cmds[3].get_motion_time()
        )
        return time

In [ ]:
print(ROBOT_STATUS)

In [ ]:
BASKET_STATUS['ready'] = 0
BASKET_STATUS['ret0'] = 1
BASKET_STATUS['w0'] = 1

In [1]:
def todo():
    todo = {}
    todo[0] = []
    todo[1] = []
    todo[2] = []
    todo[3] = []

    # 0 순위 작업
    for i in range(6):
        if (BASKET_STATUS['f{}'.format(i)] > 20):
            if (BASKET_STATUS['fin'] == basket_nc['없음']):
                todo[0].append(PickNPlace('f{}'.format(i), 'fin'))
            else:
                todo[0].append(PickNPlace('f{}'.format(i), 'w{}'.format(i)))

    # 1 순위 작업
    for i in range(6):
        if (10 < BASKET_STATUS['ready'] < 20) and BASKET_STATUS['w{}'.format(i)] == basket_nc['없음'] and BASKET_STATUS['f{}'.format(i)] == basket_nc['없음']:
            todo[1].append(PickNPlace('ready', 'w{}'.format(i)))
            # todo.append(PickNShake('ready', 'f{}'.format(i), 2))

        if (BASKET_STATUS['w{}'.format(i)] == basket_nc['빈바구니']) and (BASKET_STATUS['ready'] == basket_nc['없음']):
            todo[1].append(PickNPlace('w{}'.format(i), 'ready'))

    for j in range(3):
        if (BASKET_STATUS['ret{}'.format(j)] != basket_nc['없음']) and (BASKET_STATUS['ready'] == basket_nc['없음']):
            todo[1].append(PickNPlace('ret{}'.format(j), 'ready'))

    # 2 순위 작업
    for i in range(6):
        if (10 < BASKET_STATUS['w{}'.format(i)] < 20) and (BASKET_STATUS['f{}'.format(i)] == basket_nc['없음']):
            todo[2].append(PickNShake('w{}'.format(i), 'f{}'.format(i)))

        if (BASKET_STATUS['w{}'.format(i)] > 20) and (BASKET_STATUS['fin'] == basket_nc['없음']):
            todo[2].append(PickNPlace('w{}'.format(i), 'fin'))

        for j in range(3):
            if BASKET_STATUS['ret{}'.format(j)] != basket_nc['없음'] and BASKET_STATUS['w{}'.format(i)] == basket_nc['없음'] and BASKET_STATUS['f{}'.format(i)] == basket_nc['없음']:
                todo[2].append(PickNPlace('ret{}'.format(j), 'w{}'.format(i)))

    return todo

def next_work(todo):

    if todo[0]:
        hurryup = min(FRYING_TIME, key=FRYING_TIME.get)
        for w in todo[0]:
            if w.pick_pos_code == pos_nc[hurryup]:
                return w

    left_time = min(FRYING_TIME.values())
    if todo[1]:
        work = min(a[1], key=lambda t:t.get_time())
        if left_time - work.get_time() < -30:
            pass
        else:
            return work

    if todo[2]:
        work = min(a[2], key=lambda t:t.get_time())
        if left_time - work.get_time() < -30:
            pass
        else:
            return work
            

    return None

print(next_work(todo()))

NameError: name 'BASKET_STATUS' is not defined

In [13]:
a = 'shaking_08'

In [14]:
print(int(a[-2:]))
if int(a[-2:]) == 8:
    print('yes')
else:
    print('no')


08
yes
